<a href="https://colab.research.google.com/github/AndreaFabro/Tech_fase3/blob/main/finetuning_split_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# instalando o Hugging Face Transformers e dataset
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import json
from datasets import Dataset

# Caminho do dataset no Google Drive
INPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/split_titles_dataset_1.json'

# 1. Carregar o dataset original
with open(INPUT_PATH_DATASET, 'r', encoding='utf-8') as input_file:
    data = json.load(input_file)

# 2. Pré-processar o campo 'input' para garantir que é uma string
def preprocess_input(file):
    if isinstance(file['input'], list):  # Se for uma lista, unir os elementos em uma string
        file['input'] = " ".join(file['input'])

    return file

# Aplicar a função de pré-processamento a cada item do dataset
preprocessed_data = [preprocess_input(file) for file in data]

# 3. Converter o dataset para o formato Hugging Face
dataset = Dataset.from_list(preprocessed_data)

# Verificar o dataset processado
print(dataset)
print(dataset[0]['input'])


Dataset({
    features: ['input'],
    num_rows: 100000
})
DESCRIBE THIS PRODUCTS.
[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]

[|Content|]High quality 3 layer ballet tutu. 12 inches in length[|eContent|]


In [4]:
# Verificar os primeiros 3500 caracteres do arquivo formatado
with open(INPUT_PATH_DATASET, 'r', encoding='utf-8') as file:
    formatted_data = file.read(3500)
    print(formatted_data)


[
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]\n\n[|Content|]High quality 3 layer ballet tutu. 12 inches in length[|eContent|]"
    },
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Mog's Kittens[|eTitle|]\n\n[|Content|]Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.[|eContent|]"
    },
    {
        "input": "DESCRIBE THIS PRODUCTS.\n[|Title|] Girls Ballet Tutu Neon Blue[|eTitle|]\n\n[|Content|]Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun![|eContent|]"
    },
    {
        "input": "DESCRIBE 

In [5]:
import json

# Gerar arquivo no formato alpaca:
#    formatted_data = {
#        "instruction": instructions,
#        "input": inputs,
#        "output": outputs
#    }

def format_dataset_into_model_input(data):
    def separate_text(full_text):
        # Separar as partes do texto com base nos delimitadores fornecidos
        title_start = full_text.find("[|Title|]") + len("[|Title|]")
        title_end = full_text.find("[|eTitle|]")
        content_start = full_text.find("[|Content|]") + len("[|Content|]")
        content_end = full_text.find("[|eContent|]")

        # Instrução é a primeira linha
        instruction = full_text.split('\n')[0].strip()
        input_text = full_text[title_start:title_end].strip()
        response = full_text[content_start:content_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    for item in data:
        # Extrair o texto da chave "input"
        full_text = item["input"]
        instruction, input_text, response = separate_text(full_text)

        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(response)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, indent=4, ensure_ascii=False)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

INPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/split_titles_dataset_1.json'
OUTPUT_PATH_DATASET = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Aplicar a função de pré-processamento a cada item do dataset
preprocessed_data = [preprocess_input(file) for file in data]

# 3. Converter o dataset para o formato Hugging Face
dataset = Dataset.from_list(preprocessed_data)

# Verificar o dataset processado
print(dataset)
print(dataset[0]['input'])

# Carregar o dataset original
#with open(dataset, 'r', encoding='utf-8') as input_file:
#    data = json.load(input_file)

# Chamar a função com o dataset carregado
format_dataset_into_model_input(dataset)


Dataset({
    features: ['input'],
    num_rows: 100000
})
DESCRIBE THIS PRODUCTS.
[|Title|] Girls Ballet Tutu Neon Pink[|eTitle|]

[|Content|]High quality 3 layer ballet tutu. 12 inches in length[|eContent|]
Dataset salvo em /content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json


In [9]:
# Verificar os primeiros 500 caracteres do arquivo formatado
with open(OUTPUT_PATH_DATASET, 'r', encoding='utf-8') as file:
    formatted_data = file.read(500)
    print(formatted_data)

{
    "instruction": [
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS PRODUCTS.",
        "DESCRIBE THIS


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Escolha do modelo pré-treinado que vamos utilizar
MODEL_NAME = "EleutherAI/gpt-neo-125M"

# Carregar o tokenizer e o modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [11]:
from datasets import load_dataset

# Caminho para o arquivo JSON formatado
ALPACA_DATASET_PATH = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Carregar o dataset no formato JSON
dataset = load_dataset('json', data_files=ALPACA_DATASET_PATH)
print(dataset)
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])
print(dataset['train'][3])
print(dataset['train'][4])
print(dataset['train'][5])


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100000
    })
})
{'instruction': 'DESCRIBE THIS PRODUCTS.', 'input': 'Girls Ballet Tutu Neon Pink', 'output': 'High quality 3 layer ballet tutu. 12 inches in length'}
{'instruction': 'DESCRIBE THIS PRODUCTS.', 'input': "Mog's Kittens", 'output': 'Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.'}
{'instruction': 'DESCRIBE THIS PRODUCTS.', 'input': 'Girls Ballet Tutu Neon Blue', 'output': 'Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!'}
{'instruction': 

In [12]:
# Verificar se o número de entradas é consistente entre colunas
input_lengths = len(dataset['train']['input'])
instruction_lengths = len(dataset['train']['instruction'])
output_lengths = len(dataset['train']['output'])

print(f"Length of 'instruction': {instruction_lengths}")
print(f"Length of 'input': {input_lengths}")
print(f"Length of 'output': {output_lengths}")

# Exibir algumas amostras do dataset carregado
print(dataset)


Length of 'instruction': 100000
Length of 'input': 100000
Length of 'output': 100000
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100000
    })
})


In [13]:
from datasets import load_dataset

# Caminho para o arquivo JSON formatado
ALPACA_DATASET_PATH = '/content/drive/MyDrive/finetuning/formatted_alpaca_dataset.json'

# Carregar o dataset no formato JSON
dataset = load_dataset('json', data_files=ALPACA_DATASET_PATH)

# Definir o token de padding como o token de fim de sequência (eos_token)
tokenizer.pad_token = tokenizer.eos_token

# Ou adicionar um novo token de padding, se necessário
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize_function(batch):
    tokenized_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    # Iterar pelos exemplos no batch
    for instruction, input_text, output_text in zip(batch['instruction'], batch['input'], batch['output']):
        # Concatenar o input ao prompt (se o input não for vazio)
        prompt = instruction
        if input_text:
            prompt += " " + input_text

        # Tokenizar o prompt (entrada) e a resposta (saída)
        tokenized_input = tokenizer(prompt, truncation=True, padding='max_length', max_length=512)
        tokenized_output = tokenizer(output_text, truncation=True, padding='max_length', max_length=512)

        # Adicionar os tokens ao dicionário tokenized_batch
        tokenized_batch["input_ids"].append(tokenized_input['input_ids'])
        tokenized_batch["attention_mask"].append(tokenized_input['attention_mask'])
        tokenized_batch["labels"].append(tokenized_output['input_ids'])

    return tokenized_batch

# Aplicar a função de tokenização ao dataset com batched=True
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Verificar a saída do dataset tokenizado
print(tokenized_datasets)




Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
})


In [14]:
print(tokenized_datasets['train'][0])
print(tokenized_datasets['train'][1])
print(tokenized_datasets['train'][2])

{'instruction': 'DESCRIBE THIS PRODUCTS.', 'input': 'Girls Ballet Tutu Neon Pink', 'output': 'High quality 3 layer ballet tutu. 12 inches in length', 'input_ids': [30910, 34, 7112, 12473, 12680, 41458, 50, 13, 12002, 8528, 1616, 22792, 84, 27093, 14657, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [15]:
# Dividir o dataset em treino (90%) e validação (10%)
dataset = dataset['train'].train_test_split(test_size=0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [16]:
# Tokenizar o dataset de treino e validação
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [17]:
from transformers import DataCollatorForLanguageModeling

# Criar o collator para causal language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Definir os parâmetros de treinamento
training_args = TrainingArguments(
    output_dir='./outputs',
    evaluation_strategy="steps",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps = 5,
    max_steps = 60,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1,
    save_total_limit=2,
    save_steps=1000,
    eval_steps=1000,
    load_best_model_at_end=True,
)


# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [19]:
# Treinar o modelo
trainer.train()


Step,Training Loss,Validation Loss


TrainOutput(global_step=60, training_loss=2.941032314300537, metrics={'train_runtime': 4484.5119, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.013, 'total_flos': 125379406725120.0, 'train_loss': 2.941032314300537, 'epoch': 0.005333333333333333})

In [ ]:
# Avaliar o modelo
results = trainer.evaluate()
print(results)

In [ ]:
# Salvar o modelo treinado e o tokenizador
trainer.save_model('./my_finetuned_model')
tokenizer.save_pretrained('./my_finetuned_model')